In [91]:
import pandas as pd


In [93]:

df1 = pd.read_csv('combined_data.csv')
# df2 = pd.read_csv('som.csv')

In [94]:
df1

,raw_text,Title,header,abstract,pc_removal,wmail,urls,wurl,refs,wrefs,wnumbers,sentences,clean
0,\r\nthe subject of structural health monitorin...,http : / / a m m t i a c . a l i o n s c i e n...,http://ammtiac.alionscience.com the ammtiac qu...,NaN,\r\nthe subject of structural health monitorin...,\r\nthe subject of structural health monitorin...,['http://ammtiac.alionscience.com/quarterly\nh...,\r\nthe subject of structural health monitorin...,"\r\n[1] structural health monitoring workshop,...",\r\nthe subject of structural health monitorin...,\r\nthe subject of structural health monitorin...,['the subject of structural health monitoring ...,the subject of structural health monitoring (s...
1,\r\nthe damage identification in composites la...,Structuraldamageidentiﬁcationinlaminatedstruct...,structural damage identification in laminated ...,\r\na damage identification technique based on...,\r\nthe damage identification in composites la...,\r\nthe damage identification in composites la...,['www.elsevier.com/locate/compstruct\nto'],\r\nthe damage identification in composites la...,"\r\n[1] doebling sw, farrar cr, prime mb, shev...",\r\nthe damage identification in composites la...,\r\nthe damage identification in composites la...,['the damage identification in composites lami...,the damage identification in composites lamina...
2,\r\nstructural optimization has become one of ...,JournalofConstructionalSteelResearch,journal of constructional steel research 65 (2...,"\r\nin this paper, a particle swarm optimizer ...",\r\nstructural optimization has become one of ...,\r\nstructural optimization has become one of ...,[],\r\nstructural optimization has become one of ...,"\r\n[1] kaveh a, farahmand azar b, talatahari ...",\r\nstructural optimization has become one of ...,\r\nstructural optimization has become one of ...,['structural optimization has become one of th...,structural optimization has become one of the ...


### convert df to json 

In [95]:
from llama_index.core import Document # type: ignore

train = [
    Document(
        text=row['clean'],
        description= row['abstract'],
        metadata={
            'Title': row['Title'],
            'urls': row['urls'],
            'refs': row['refs'],
            'authors': row['authors']
        }
    )
    for _, row in df1.iterrows()
]
# eval = [
#     Document(
#         text=row['clean'],
#         description= row['abstract'],
#         metadata={
#             'Title': row['Title'],
#             'urls': row['urls'],
#             'refs': row['refs']
#         }
#     )
#     for _, row in df2.iterrows()
# ]

### allow async loops

In [49]:
import nest_asyncio
nest_asyncio.apply()

### Generate Corpus

In [101]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import MetadataMode

TRAIN_FILES = train
VAL_FILES = eval

In [118]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from llama_index.core.node_parser import LangchainNodeParser
from llama_index.core.schema import MetadataMode 

def load_corpus(docs, verbose=False):
    if verbose:
        print(f"Loading files in {docs}")
    print(docs)
    if verbose:
        print(f"Loaded {len(docs)} docs")
    parser = LangchainNodeParser(RecursiveCharacterTextSplitter())
    nodes = parser.get_nodes_from_documents(docs)
    if verbose:
        print(f"Parsed {len(nodes)} nodes")
    return nodes

train_nodes = load_corpus(TRAIN_FILES, verbose=True)
val_nodes = load_corpus(VAL_FILES, verbose=True)

Loading files in [Document(id_='6b206d8b-fc92-4f4c-944e-7f58e8606099', embedding=None, metadata={'Title': 'http : / / a m m t i a c . a l i o n s c i e n c e . c o m / q u a r t e r l y h t t p : / / a m m t i a c . a l i o n s c i e n c e . c o m / q u a r t e r l y', 'urls': "['http://ammtiac.alionscience.com/quarterly\\nhttp://ammtiac.alionscience.com/quarterly\\nthe', 'http://ammtiac.alionscience.com/quarterly\\nhttp://ammtiac.alionscience.com/quarterly\\nhttp://ammtiac.alionscience.com', 'http://ammtiac.alionscience.com/quarterly\\nhttp://ammtiac.alionscience.com/quarterly\\nthe', 'http://ammtiac.alionscience.com/quarterly\\nhttp://ammtiac.alionscience.com/quarterly']", 'refs': '\r\n[1] structural health monitoring workshop, sponsored by army, navy,\r\nand air force, administered by the advanced materials, manufacturing\r\n& testing information analysis center (ammtiac), november 18-20,\r\n2008, austin, tx.\r\n[2] hall, s. r., “the effective management and use of structural health

### Generate synthetic queries

In [121]:
%pip install llama-index-llms-openai -q
%pip install llama-index-embeddings-openai -q
%pip install llama-index-finetuning -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

d:\Projects\LLM\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setting up Llama2

In [2]:
%pip install llama-index-llms-huggingface -q -U
%pip install llama-index-embeddings-huggingface -q -U
!pip install llama-index ipywidgets -q -U

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


'pip' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


from IPython.display import Markdown, display

In [ ]:
import torch
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

# Model names (make sure you have access on HF)
LLAMA2_7B = "meta-llama/Llama-2-7b-hf"
LLAMA2_7B_CHAT = "meta-llama/Llama-2-7b-chat-hf"
LLAMA2_13B = "meta-llama/Llama-2-13b-hf"
LLAMA2_13B_CHAT = "meta-llama/Llama-2-13b-chat-hf"
LLAMA2_70B = "meta-llama/Llama-2-70b-hf"
LLAMA2_70B_CHAT = "meta-llama/Llama-2-70b-chat-hf"

selected_model = LLAMA2_7B_CHAT

SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<>\n" + SYSTEM_PROMPT + "<>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
    quantization_config={}
)

### dataset generation

In [ ]:
train_dataset = generate_qa_embedding_pairs(train_nodes, llm=llm)
train_dataset.save_json("train_dataset.json")
train_dataset = EmbeddingQAFinetuneDataset.from_json("train_dataset.json")
val_dataset = generate_qa_embedding_pairs(val_nodes, llm=llm)
val_dataset.save_json("val_dataset.json")       
val_dataset = EmbeddingQAFinetuneDataset.from_json("val_dataset.json")

### Fine-tuning BAAI/bge-small-en-v1.5

In [79]:
%pip install sentence_transformers -q -U

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset, # Dataset to be trained on
    model_id="BAAI/bge-small-en-v1.5", # HuggingFace reference to base embeddings model
    model_output_path="llama_model_v1", # Output directory for fine-tuned embeddings model
    val_dataset=val_dataset, # Dataset to validate on
    epochs=2 # Number of Epochs to train for
)

In [82]:
%pip install llama-index-embeddings-huggingface -q

Note: you may need to restart the kernel to use updated packages.


In [84]:
from llama_index.core import ServiceContext, set_global_service_context
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceWindowNodeParser

# window node parser
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=6,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
# base Query Engine LLM
# llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

# fine-tuned Embeddings model
embed_model = HuggingFaceEmbedding(
   model_name="BAAI/bge-small-en"
)

# # base Embeddings model
# embed_model_base = HuggingFaceEmbedding(
#     model_name="BAAI/bge-small-en"
# )

# fine-tuned ServiceContext
ctx = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

# # base ServiceContext
# ctx_base = ServiceContext.from_defaults(
#     llm=llm,
#     embed_model=embed_model_base
# )


nodes = node_parser.get_nodes_from_documents(TRAIN_FILES)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en
Load pretrained SentenceTransformer: BAAI/bge-small-en


modules.json: 100%|██████████| 349/349 [00:00<00:00, 139kB/s]
d:\Projects\LLM\.conda\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sai18\AppData\Local\llama_index\models--BAAI--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sentence_transformers.json: 100%|██████████| 124/124

INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


NameError: name 'llm' is not defined

In [78]:
nodes

[TextNode(id_='f499515b-0463-4179-92df-c6c02d6992c5', embedding=None, metadata={'window': 'structural optimization has become one of the most active\nbranches of structural engineering in the last decade.  the large\nnumber of design variables, large size of the search space and\nthe control of a great number of design constraints are major\npreventive factors in performing optimum design in a reasonable\ntime.  despite these facts, designers and owners have always\ndesired to have optimal structures [1].  recently, authors have presented an efficient continuous\noptimization algorithm, so-called heuristic particle swarm ant\ncolony optimization (hpsaco) [2], for truss problems, which\nis low time consuming and highly reliable.  in hpsaco, the\npsopc algorithm (a hybrid pso with passive congregation [3])\nis combined with the ant colony algorithm and harmony search\napproach.  the hpsaco applies the psopc for global optimization\nand an ant colony approach is employed as a local search

In [ ]:
from llama_index import VectorStoreIndex
sentence_index = VectorStoreIndex(nodes, service_context=ctx)

In [ ]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=3,
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)

In [ ]:
window_response = query_engine.query("How do camelid genetics influence wool quality?")

In [ ]:
window_response.response